# CA3

In [1]:
import numpy as np, torch, matplotlib.pyplot as plt, pandas as pd, torch.multiprocessing as mp
#%inline


# 1) Solving with different methods
## Make the optimisation algorithms
### Gradient decent

In [2]:
# regular gradient descent
class GD(torch.optim.Optimizer):
    
    def __init__(self, params, lr):
        torch.optim.Optimizer.__init__(self, params, defaults=dict(lr=lr))
        
    def step(self):
        
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                else:
                    
                    # W -= learning_rate*gradient
                    p.data.add_(-group['lr']*p.grad.data)

### Perturbed gradient descent

In [3]:
class PGD(torch.optim.Optimizer):
    
    def __init__(self, params, lr):
        torch.optim.Optimizer.__init__(self, params, defaults=dict(lr=lr))
        
    def step(self):
        
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                else:
                    
                    # W -= learning_rate*gradient
                    p.data.add_(-group['lr']*(p.grad.data*np.random.uniform(0.90, 1.1)))

### Stochastic variance reduced gradient decsent

In [4]:
class SVRG(torch.optim.Optimizer):

    def __init__(self, params, lr=1e-4, freq =10):

        defaults = dict(lr=lr, freq=freq)
        self.counter = 0
        self.counter2 = 0
        self.flag = False
        super(SVRG, self).__init__(params, defaults)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            freq = group['freq']
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                param_state = self.state[p]
                
                if 'large_batch' not in param_state:
                    buf = param_state['large_batch'] = torch.zeros_like(p.data)
                    buf.add_(d_p) #add first large, low variance batch
                    #need to add the second term in the step equation; the gradient for the original step!
                    buf2 = param_state['small_batch'] = torch.zeros_like(p.data)

                buf = param_state['large_batch']
                buf2 = param_state['small_batch']

                if self.counter == freq:
                    buf.data = d_p.clone() #copy new large batch. Begining of new inner loop
                    temp = torch.zeros_like(p.data)
                    buf2.data = temp.clone()
                    
                if self.counter2 == 1:
                    buf2.data.add_(d_p) #first small batch gradient for inner loop!

                #dont update parameters when computing large batch (low variance gradients)
                if self.counter != freq and self.flag != False:
                    p.data.add_(-group['lr'], (d_p - buf2 + buf) )

        self.flag = True #rough way of not updating the weights the FIRST time we calculate the large batch gradient
        
        if self.counter == freq:
            self.counter = 0
            self.counter2 = 0

        self.counter += 1    
        self.counter2 += 1

        return loss

### Block coordinate descent

In [5]:
class BCD(torch.optim.Optimizer):
    
    def __init__(self, params, lr):
        torch.optim.Optimizer.__init__(self, params, defaults=dict(lr=lr))
        
    def step(self):
        
        for group in self.param_groups:
            p = group['params'][np.random.randint(0,len(group['params']))]
            p.data.add_(-group['lr']*p.grad.data)


### Stochastic gradient descent
This is built into PyTorch.

## Training function

In [6]:
# personal trainer
def train(net, datai, datao, epo=100, lr=1e-3, testing_percent=0.1, method='sgd'):
    net.cuda()
    # number of testing data points
    n = int(datai.shape[0]*testing_percent)
    
    # testing data
    itst = datai[:n, :]
    otst = datao[:n, :]
    
    # training data
    itrn = datai[n:, :]
    otrn = datao[n:, :]
    
    # give the net a training log
    net.ltrn = list()
    net.ltst = list()
    
    # training method
    if method == "sgd":
        opt = torch.optim.SGD(net.parameters(), lr=lr)
    elif method == "gd":
        opt = GD(net.parameters(), lr=lr)
    elif method == "pgd":
        opt = PGD(net.parameters(), lr=lr)
    elif method == 'svrg':
        opt = SVRG(net.parameters(), lr=lr)
    elif method == 'bcd':
        opt = BCD(net.parameters(), lr=lr)
    elif method == 'adam':
        opt = torch.optim.Adam(net.parameters(), lr=lr)
        
    
    # loss function
    #lf = torch.nn.MSELoss()
    
    # time to train!
    for e in range(epo):
        
        # zero the gradients
        opt.zero_grad()
        
        # testing loss
        ltst = torch.mean(torch.pow(torch.norm(net(itst) - otst, 2, dim=1), 2))
        
        # training loss
        ltrn = torch.mean(torch.pow(torch.norm(net(itrn) - otrn, 2, dim=1), 2))
        
        # record losses
        net.ltst.append(ltst.item())
        net.ltrn.append(ltrn.item())
        
        # print progress
        #print("Method: {}; Episode: {}; Training: {}; Testing: {}".format(method, e, net.ltrn[-1], net.ltst[-1]))
        
        # backpropagate erros
        ltrn.backward()
        
        # update weights
        opt.step()
    
    return net

## Get the data

In [7]:
'''
load_txt = 'household_power_consumption'
household = np.load(load_txt +'.npy').item()
N = len(household['Date'])
ArrayShape = (N,1)
y  = np.asarray(household['Global_active_power']).reshape(ArrayShape)
x1 = np.asarray(household['Global_reactive_power']).reshape(ArrayShape)
x2 = np.asarray(household['Voltage']).reshape(ArrayShape)
x3 = np.asarray(household['Global_intensity']).reshape(ArrayShape)
x4 = np.asarray(household['Sub_metering_1']).reshape(ArrayShape)
x5 = np.asarray(household['Sub_metering_2']).reshape(ArrayShape)
x6 = np.asarray(household['Sub_metering_3']).reshape(ArrayShape)
x =  np.concatenate((x1, x2, x3, x4, x5, x6), axis = 1)
'''
load_txt = 'CrimesNp'
data = np.load(load_txt + ".npy")
y = data[:,data.shape[1]-1]
y = y.reshape(y.shape[0],1)
x = data[:,:data.shape[1]-2]


In [8]:
# input and output dimensions
di, do = x.shape[1], x.shape[1]

# input and output data to tensors
x = torch.Tensor(x).cuda()
y = torch.Tensor(y).cuda()
print(x.shape)
print(y.shape)

torch.Size([1994, 98])
torch.Size([1994, 1])


## Train the deep neural network with different methods

In [9]:
'''
# single network

# number of hidden nodes
hn = 100

net = torch.nn.Sequential(
        torch.nn.LayerNorm(di),  # normalise x_i
        torch.nn.Linear(di, hn), # W1
        torch.nn.Sigmoid(),      # s
        torch.nn.Linear(hn, hn), # W2
        torch.nn.Sigmoid(),      # s
        torch.nn.Linear(hn, do)  # w3
)

train(net, x, y, epo=1000, lr=1e-3, testing_percent=0.1, method='sgd')
'''

"\n# single network\n\n# number of hidden nodes\nhn = 100\n\nnet = torch.nn.Sequential(\n        torch.nn.LayerNorm(di),  # normalise x_i\n        torch.nn.Linear(di, hn), # W1\n        torch.nn.Sigmoid(),      # s\n        torch.nn.Linear(hn, hn), # W2\n        torch.nn.Sigmoid(),      # s\n        torch.nn.Linear(hn, do)  # w3\n)\n\ntrain(net, x, y, epo=1000, lr=1e-3, testing_percent=0.1, method='sgd')\n"

In [10]:
# number of hidden nodes
hn = 150
nets = []
for i in range(6):
        torch.manual_seed(0)
        nets.append(torch.nn.Sequential(
        torch.nn.LayerNorm(di),  # normalise x_i
        torch.nn.Linear(di, hn), # W1
        torch.nn.Sigmoid(),      # s
        torch.nn.Linear(hn, hn), # W2
        torch.nn.Sigmoid(),      # s
        torch.nn.Linear(hn, do)  # w3
        ))
for i, net in enumerate(nets):
    train(net, x, y, epo=200, lr=1e-3, testing_percent=0.1, method=['gd', 'pgd', 'sgd', 'svrg', 'bcd', 'adam'][i])

In [11]:
# networks in parallel
'''

# number of hidden nodes
hn = 100

# DEEEEEP neural network
nets = [(
    torch.nn.Sequential(
        torch.nn.LayerNorm(di),  # normalise x_i
        torch.nn.Linear(di, hn), # W1
        torch.nn.Sigmoid(),      # s
        torch.nn.Linear(hn, hn), # W2
        torch.nn.Sigmoid(),      # s
        torch.nn.Linear(hn, do)  # w3
    ), x, y, 5000, 1e-2, 0.1, method
    ) for method in ['gd', 'pgd', 'sgd', 'svrg', 'bcd']]

nets = mp.Pool(mp.cpu_count()).starmap(train, nets)
'''

"\n\n# number of hidden nodes\nhn = 100\n\n# DEEEEEP neural network\nnets = [(\n    torch.nn.Sequential(\n        torch.nn.LayerNorm(di),  # normalise x_i\n        torch.nn.Linear(di, hn), # W1\n        torch.nn.Sigmoid(),      # s\n        torch.nn.Linear(hn, hn), # W2\n        torch.nn.Sigmoid(),      # s\n        torch.nn.Linear(hn, do)  # w3\n    ), x, y, 5000, 1e-2, 0.1, method\n    ) for method in ['gd', 'pgd', 'sgd', 'svrg', 'bcd']]\n\nnets = mp.Pool(mp.cpu_count()).starmap(train, nets)\n"

In [12]:
'''
# train more
nets = [(
    net, x, y, 5000, 1e-2, 0.1, method
    ) for net, method in zip(nets, ['gd', 'pgd', 'sgd', 'svrg', 'bcd', 'adam'])]
nets = mp.Pool(mp.cpu_count()).starmap(train, nets)
'''

"\n# train more\nnets = [(\n    net, x, y, 5000, 1e-2, 0.1, method\n    ) for net, method in zip(nets, ['gd', 'pgd', 'sgd', 'svrg', 'bcd', 'adam'])]\nnets = mp.Pool(mp.cpu_count()).starmap(train, nets)\n"

## Visualisation

In [13]:
%matplotlib
fig, ax = plt.subplots(6, sharex=True)
meth = ['gd', 'pgd', 'sgd', 'svrg', 'bcd','adam']
for i, net in enumerate(nets):
    ax[i].plot(net.ltrn, "k-", alpha=0.4)
    ax[i].plot(net.ltst, "k--", alpha=0.4)
    #print(i)
    if i == 0:
        ax[i].set_title(load_txt)
        ax[i].set_ylabel("Loss("+meth[i] + ")")
    else:
        ax[i].set_ylabel("("+meth[i] + ")")
    #ax[i].set_title(meth[i])
    #ax[i].set_ylim(0,4)
    #ax[i].set_xscale('log')
    ax[i].grid(linestyle='-')
ax[-1].set_xlabel('Episode')
ax[0].legend(['Training', 'Testing'])
#fig.tight_layout()
fig.savefig(load_txt +'.pdf', bbox_inches='tight')
plt.show()

Using matplotlib backend: TkAgg


# 2) Comparing methods
Compare these solvers in terms complexity of hyper-parameter tunning, conver-
gence time, convergence rate (in terms of # outer-loop iterations), and memory

requirement

Hyper parameter tuning:

- GD: higher learning rate makes training faster but more erratic, lower learning rate make training slower but more reliable.
- PGD: Similar to GD.
- SGD: Similar to GD, we can increase momentum to potentially evade local minimia.
- SVRG: Similar to GD.
- BCD: Similar to GD.

Convergence time:

- GD, PGD, SGD, SVRG seems to converge almost at the same time, but base on the Adam optimizer, we can see that all of them seems to converge to a local minima.
- BCD has a slower convergence time in this case 

Convergence Rate:

- Similar convergence rate for GD, PGD, SGD, SVRG except for BCD

Memory:

- GD memory usage is huge
- PGD is similar
- SGD is better because it takes batches
- SVRG easier than gradient descent
- BCD similar to GD


